In [ ]:
import spacy
import glob
import pandas as pd

nlp = spacy.load('en_core_web_sm')


path = './Biographies/*'
files = sum([glob.glob(dir+'/*.txt') for dir in glob.glob(path)],[])

filenames = [file.split('/')[3] for file in files]

ents_spacy=[]
for file in files:
    with open(file) as infile:
        file_content =  infile.read()
        doc = nlp(file_content)
        ents_spacy.append(doc.ents)

data_spacy = list(zip(filenames,ents))
df_spacy = pd.DataFrame(data_spacy, columns = ['filename','ents'])
print(df_spacy.head())

In [4]:
import stanza
stanza.download('en')
nlp = stanza.Pipeline(lang="en") 

ents_stanza=[]
for file in files:
    with open(file) as infile:
        file_content =  infile.read()
        doc = nlp(file_content)
        ents_stanza.append(doc.ents)

data_stanza = list(zip(filenames,ents))
df_stanza = pd.DataFrame(data, columns = ['filename','ents'])
print(df_stanza.head())

/home/marina/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 22:56:00 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-11 22:56:00 INFO: Downloading default packages for language: en (English) ...
2024-06-11 22:56:02 INFO: File exists: /home/marina/stanza_resources/en/default.zip
2024-06-11 22:56:05 INFO: Finished downloading models and saved to /home/marina/stanza_resources
2024-06-11 22:56:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-11 22:56:06 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-11 22:56:07 INFO: Loading these models for language: en (English):
| Processo

In [17]:
### all together ########################
import stanza
import spacy
import glob
import pandas as pd

#stanza.download('en')
nlp_stanza = stanza.Pipeline(lang="en") 
nlp_spacy = spacy.load('en_core_web_sm')

path = './Biographies/*'
files = sum([glob.glob(dir+'/*.txt') for dir in glob.glob(path)],[]) # a commenter pour tout faire
files = files[:3]

data = []
for file in files:
    filename = file.split('/')[-1]
    print(file)
    with open(file, 'r') as f:
        content = f.read()

        doc_stanza = nlp_stanza(content)
        ents_stanza = [(ent.text, ent.type) for ent in doc_stanza.ents]
    
        doc_spacy = nlp_spacy(content)
        ents_spacy = [(ent.text, ent.label_) for ent in doc_spacy.ents]
    
        data.append({'file_name': filename,
                    'ents_stanza': ents_stanza,
                    'ents_spacy': ents_spacy})

df = pd.DataFrame(data)
df.head()

2024-06-11 23:57:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-11 23:57:09 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-11 23:57:10 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-06-11 23:57:10 INFO: Using device: cpu
2024-06-11 23:57:10 INFO: Loading: tokenize
2024-06-11 23:57:10 INFO: Loading: mwt
2024-06-11 23:57:10 INFO: Loading: pos
2024

./Biographies/Biographies_Journalists/AquileoJ.Echeverría_Journalists.txt
./Biographies/Biographies_Journalists/ThomasHenryShadwellClerke_Journalists.txt
./Biographies/Biographies_Journalists/Journalist_Journalists.txt


,file_name,ents_stanza,ents_spacy
0,AquileoJ.Echeverría_Journalists.txt,"[(Aquileo J. Echeverría, PERSON), (May 22, 186...","[(Aquileo J. Echeverría, PERSON), (May 22, 186..."
1,ThomasHenryShadwellClerke_Journalists.txt,"[(Thomas Henry Shadwell Clerke, PERSON), (KH, ...","[(Thomas Henry Shadwell Clerke, PERSON), (KH, ..."
2,Journalist_Journalists.txt,"[(Matthew C. Nisbet, PERSON), (Walter Lippmann...","[(Matthew C. Nisbet, PERSON), (Walter Lippmann..."
